In [72]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from pymatgen.io.cif import CifWriter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [73]:
# HfNiSn
mp_key = "fnNhNeZr2KXTJ64m73UhQ93prbDkSyqH"
mp_id = "mp-924128"

In [74]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

## Fichier CIF

In [75]:
with MPRester(mp_key) as m:
    
    cif_data = m.get_structure_by_material_id(mp_id)
    print("Téléchargement ok")
structure_conv = SpacegroupAnalyzer(cif_data).get_conventional_standard_structure()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Téléchargement ok


In [76]:
view = quick_view(cif_data, conventional = 'True')
display(view)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [77]:
symetrie = SpacegroupAnalyzer(cif_data).get_symmetry_operations()

#opérations de symétrie
s_1 = symmops[31]
s_2 = symmops[5]
s_3 = symmops[47]

#Coordonnées atomes
coord_Hf = [0.0, 0.0, 0.0]
coord_Ni = [0.25, 0.25, 0.25]
coord_Sn = [0.5, 0.5, 0.5]

#application symétrie 
Hf_sym = s_1.operate(coord_Hf);
Ni_sym = s_2.operate(coord_Ni);
Sn_sym = s_3.operate(coord_Sn);

## Symétrie 1 : Plan miroir

In [78]:
print("L'atome de départ est le Hf et a comme coordonnées: {}\n".format(coord_Hf))
print("On lui applique une symétrie de plan miroir (z,y,x) :\n")
print(s_1.rotation_matrix , "\n")

print("Le vecteur de translation est : ", s_1.translation_vector, "\n")
print("L'atome d'arrivé a comme coordonnées : {}\n".format(Hf_sym))

v1 = quick_view(cif_data)
display(v1)
v1.script("draw symop 32 {atomno=1}")

L'atome de départ est le Hf et a comme coordonnées: [0.0, 0.0, 0.0]

On lui applique une symétrie de plan miroir (z,y,x) :

[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]] 

Le vecteur de translation est :  [0.5 0.5 0. ] 

L'atome d'arrivé a comme coordonnées : [0.5 0.5 0. ]



JsmolView(layout=Layout(align_self='stretch', height='400px'))

## Symétrie 2 : Symétrie d'axe 2 ( z , -x-y-z , x )

In [79]:
print("L'atome de départ est le Ni et a comme coordonnées: {}\n".format(coord_Ni))
print("On lui applique une symétrie d'axe 2 ( z, -x-y-z , x ) :\n")
print(s_2.rotation_matrix , "\n")

print("Le vecteur de translation est : ", s_2.translation_vector, "\n")
print("L'atome d'arrivé a comme coordonnées : {}\n".format(Ni_sym))

v2 = quick_view(cif_data)
display(v2)
v2.script("draw symop 6 {atomno=7}")

L'atome de départ est le Ni et a comme coordonnées: [0.25, 0.25, 0.25]

On lui applique une symétrie d'axe 2 ( z, -x-y-z , x ) :

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]] 

Le vecteur de translation est :  [0. 0. 0.] 

L'atome d'arrivé a comme coordonnées : [0.25 0.25 0.25]



JsmolView(layout=Layout(align_self='stretch', height='400px'))

## Symetrie 3 : Roto-inversion ( y , -x-y-z , x )

In [80]:
print("L'atome de départ est le Sn et a comme coordonnées: {}\n".format(coord_Sn))
print("On lui applique une roto-inversion ( y, -x-y-z , x ) :\n")
print(s_3.rotation_matrix , "\n")

print("Le vecteur de translation est : ", s_3.translation_vector, "\n")
print("L'atome d'arrivé a comme coordonnées : {}\n".format(Sn_sym))

v3 = quick_view(cif_data)
v3.script("draw symop 48 {atomno=3}")
display(v3)


L'atome de départ est le Sn et a comme coordonnées: [0.5, 0.5, 0.5]

On lui applique une roto-inversion ( y, -x-y-z , x ) :

[[-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0. -1.  0.]] 

Le vecteur de translation est :  [0.5 0.5 0. ] 

L'atome d'arrivé a comme coordonnées : [ 0.   1.  -0.5]



JsmolView(layout=Layout(align_self='stretch', height='400px'))